# Inclusive and Exclusive Inputs

Inclusive and Exclusive Inputs refers to the [2.4.3. Inclusive and Exclusive Inputs](https://www.commonwl.org/user_guide/topics/inputs.html#inclusive-and-exclusive-inputs) paragraph of the _2.4. Inputs_ chapter of the [Common Workflow Language User Guide](https://www.commonwl.org/user_guide/index.html#).

It shows also how the library is able to handle and convert `record` types.

## 1. Parsing

Convert the CWL document, from its YAML representation, to the CWL DOM via [cwl-utils APIs](https://github.com/common-workflow-language/cwl-utils):

In [ ]:
from cwl_utils.parser import load_document_by_yaml, save
import yaml

with open('./workflows/inclusive-exclusive-inputs.cwl') as f:
    cwl_content = yaml.load(f, Loader=yaml.SafeLoader)

cwl = load_document_by_yaml(yaml=cwl_content, uri="io://", load_all=True)

print(
    yaml.dump(
        save(
            val=cwl,
            relative_uris=False
        )
    )
)

baseCommand: echo
class: CommandLineTool
cwlVersion: v1.2
id: io://
inputs:
- id: io:/#dependent_parameters
  type:
    fields:
    - inputBinding:
        prefix: -A
      name: io:/#dependent_parameters/dependent_parameters/itemA
      type: string
    - inputBinding:
        prefix: -B
      name: io:/#dependent_parameters/dependent_parameters/itemB
      type: string
    name: io:/#dependent_parameters/dependent_parameters
    type: record
- id: io:/#exclusive_parameters
  type:
  - fields:
    - inputBinding:
        prefix: -C
      name: io:/#exclusive_parameters/itemC/itemC
      type: string
    name: io:/#exclusive_parameters/itemC
    type: record
  - fields:
    - inputBinding:
        prefix: -D
      name: io:/#exclusive_parameters/itemD/itemD
      type: string
    name: io:/#exclusive_parameters/itemD
    type: record
outputs:
- id: io:/#example_out
  type: stdout
stdout: output.txt



# 2. Inputs conversion

Once the document is parsed, invoke the `cwl2ogc` APIs to convert the CWL inputs to the OGC JSON format:

In [2]:
from cwl2ogc import BaseCWLtypes2OGCConverter
import json

converter = BaseCWLtypes2OGCConverter(cwl)

inputs = converter.to_ogc(cwl.inputs)

print(json.dumps(inputs, indent=2))

{
  "#dependent_parameters": {
    "schema": {
      "type": "object",
      "properties": {
        "itemA": {
          "type": "string"
        },
        "itemB": {
          "type": "string"
        }
      },
      "required": [
        "itemA",
        "itemB"
      ]
    }
  },
  "#exclusive_parameters": {
    "schema": {
      "nullable": false,
      "anyOf": [
        {
          "type": "object",
          "properties": {
            "itemC": {
              "type": "string"
            }
          },
          "required": [
            "itemC"
          ]
        },
        {
          "type": "object",
          "properties": {
            "itemD": {
              "type": "string"
            }
          },
          "required": [
            "itemD"
          ]
        }
      ]
    }
  }
}


# 2. Outputs conversion

Users can reuse the `BaseCWLtypes2OGCConverter` instance to convert the CWL outputs to the OGC JSON format:

In [3]:
outputs = converter.to_ogc(cwl.outputs)

print(json.dumps(outputs, indent=2))

{
  "#example_out": {
    "schema": {
      "type": "string"
    }
  }
}
